# 🧠 에이전트에 메모리 추가하기


## 🚧 **공사 중 (Under Construction)** 🚧

> 이 문서는 현재 업데이트 작업 중입니다. 실습을 권장하지 않습니다.  
> 아래의 내용은 현재 올바로 동작하지 않으며 언제든 변경될 수 있습니다.  
> 정비가 완료되면 이 안내는 사라집니다.  
> 불편을 드려 죄송합니다. 🙏


> **개요**  
> `ContextProvider`를 구현하고 에이전트에 연결하여 메모리 기능을 추가하는 방법을 알아봅니다.

---

> **⚠️ 중요**  
> 모든 에이전트 유형이 `ContextProvider`를 지원하지는 않습니다.  
> 이 과정에서는 `ContextProvider`를 지원하는 **ChatAgent**를 사용합니다.

---

## 🔧 ContextProvider 생성

> **정의**  
> `ContextProvider`는 `ChatAgent`의 `AgentThread`와 연결할 수 있는 상속 가능한 추상 클래스입니다.

### 🎯 주요 기능

ContextProvider를 통해 다음과 같은 작업을 수행할 수 있습니다:

- ✅ 에이전트가 내부 추론 서비스를 호출하기 **전/후**에 사용자 지정 로직 실행
- ✅ 에이전트가 내부 추론 서비스를 호출하기 **전**에 추가적인 컨텍스트 제공
- ✅ 에이전트에게 제공되거나 에이전트가 생성하는 **모든 메시지 검사**

---

### 📋 호출 전/후 이벤트

`ContextProvider` 클래스에는 에이전트가 내부 추론 서비스를 호출하기 전/후에 사용자 지정 로직을 실행하기 위해 재정의할 수 있는 2개의 메서드가 있습니다.

| 메서드 | 호출 시점 | 역할 |
|--------|----------|------|
| **invoking** | 추론 서비스 호출 **전** | • `context` 객체를 반환하여 추가 컨텍스트 제공<br>• 지침(instruction), 도구(tool), 메시지 추가 가능 |
| **invoked** | 추론 서비스 호출 **후** | • 요청/응답 메시지 검사<br>• ContextProvider의 상태 업데이트 |

---

### 💾 직렬화

#### 🔄 ContextProvider 생명주기

`ContextProvider` 인스턴스는 다음 시점에 생성되고 `AgentThread`에 연결됩니다:
- 스레드가 생성될 때
- 스레드가 직렬화된 상태에서 재개(Resume)될 때

#### 🗂️ 상태(State) 관리

ContextProvider 인스턴스는 에이전트 호출 간에 유지되어야 하는 **자체 상태**를 가질 수 있습니다.

#### 📦 직렬화 구현 요구사항

스레드를 계속 유지하려면:
1. ✅ `ContextProvider` 클래스에 **직렬화 기능** 구현
2. ✅ 스레드 재개 시 직렬화된 데이터에서 상태를 복원할 수 있는 **생성자** 제공

---

## 💻 샘플 ContextProvider 구현 예시

> **실습 시나리오**  
> 사용자의 이름과 나이를 기억하고 각 호출 전에 에이전트에 제공하는 사용자 지정 메모리 구성 요소를 만듭니다.

### 📝 1단계: 메모리 모델 클래스 생성

먼저 메모리를 저장할 모델 클래스를 만듭니다.

In [1]:
from pydantic import BaseModel

class UserInfo(BaseModel):
    name: str | None = None
    age: int | None = None

---

### 🔨 2단계: ContextProvider 구현

이제 `ContextProvider`를 구현하여 메모리를 관리합니다.

#### 🎯 UserInfoMemory 클래스 동작

`UserInfoMemory` 클래스는 다음과 같은 동작을 포함합니다:

| 순서 | 동작 | 설명 |
|-----|------|------|
| 1️⃣ | **정보 추출** | 채팅 클라이언트를 사용하여 매 실행이 끝날 때, 새 메시지가 스레드에 추가되면 사용자 메시지에서 사용자의 이름과 나이를 찾음 |
| 2️⃣ | **컨텍스트 제공** | 각 호출 전에 에이전트에게 현재 모든 메모리 정보를 제공 |
| 3️⃣ | **정보 요청** | 가용한 메모리가 없다면, 에이전트가 사용자에게 누락된 정보를 요청하고 정보가 제공될 때까지 어떠한 질문에도 답변하지 않도록 함 |
| 4️⃣ | **직렬화** | 스레드 상태의 일부로 메모리를 영구 저장할 수 있도록 직렬화를 구현 |

In [4]:
from typing import Any
from agent_framework import AgentSession, BaseContextProvider, SessionContext

class UserPreferenceProvider(BaseContextProvider):
    def __init__(self) -> None:
        super().__init__("user-preferences")

    async def before_run(
        self,
        *,
        agent: Any,
        session: AgentSession,
        context: SessionContext,
        state: dict[str, Any],
    ) -> None:
        if favorite := state.get("favorite_food"):
            context.extend_instructions(self.source_id, f"User's favorite food is {favorite}.")

    async def after_run(
        self,
        *,
        agent: Any,
        session: AgentSession,
        context: SessionContext,
        state: dict[str, Any],
    ) -> None:
        for message in context.input_messages:
            text = (message.text or "") if hasattr(message, "text") else ""
            if isinstance(text, str) and "favorite food is" in text.lower():
                state["favorite_food"] = text.split("favorite food is", 1)[1].strip().rstrip(".")

---

## 🚀 에이전트와 함께 ContextProvider 사용하기

### 📌 설정 방법

사용자 지정 `ContextProvider`를 사용하려면:

1. ✅ 에이전트 생성 시 인스턴스화된 `ContextProvider` 객체를 제공
2. ✅ `ChatAgent` 생성 시 `context_providers` 매개변수에 메모리 구성 요소를 전달

> **💡 핵심**  
> `context_providers` 매개변수를 통해 메모리 구성 요소를 에이전트에 연결합니다.

In [10]:
import asyncio
from agent_framework import Agent, InMemoryHistoryProvider
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential  # 동기 버전 사용!

async def main():

    # AzureOpenAIChatClient는 async with 지원 안함!
    chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())
    
    audit = InMemoryHistoryProvider("user-preferences", load_messages=False, store_context_messages=True)
    

    # 핵심: context_provider (단수!) + async with ChatAgent
    async with Agent(
        client=chat_client,
        instructions="당신은 친절한 어시스턴트입니다",
        context_providers=[audit]
    ) as agent:
        # Create a new thread for the conversation
        thread = agent.create_session()

        result2 = await agent.run("Remember that I prefer vegetarian food.", thread=thread)
        print("결과 : " + result2.text)
        print("=" * 70)

        # 메모리 확인
        print()
        print(f"✅ MEMORY - 사용자 정보: {audit.favorite if hasattr(audit, 'favorite') else '없음'}")

await main()

결과 : Got it! I’ll remember that you prefer vegetarian food and will give you vegetarian-friendly suggestions and information from now on. If you have any specific dietary restrictions or preferences within vegetarianism (like vegan, ovo-lacto, etc.), feel free to let me know!
결과 : 알려주셔서 감사합니다! 20살이시군요. 혹시 도움 필요한 것이 있으신가요? 나이나 관심사에 맞는 추천이나 조언이 필요하시다면 언제든 말씀해 주세요. 😊

✅ MEMORY - 사용자 정보: 없음
